## 3. File Listener and Database Upload
#### Write a Python script that acts as a file listener, monitoring a specific directory for file uploads. Upon detecting a new file, perform the upload of its content into a database. Use any database of your choice. Provide error handling and ensure that the script runs continuously.

In [1]:
!pip install mysql-connector-python
!pip install watchdog

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [ ]:
## Watchdo :- A library wich is used for listening to and monitoring system events like changes if happen.

In [2]:
import os
import time
import mysql.connector
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler
import shutil

In [ ]:
## Observer : allows you to watch for changes in directories.

## FileSystemEventHandler: provides methods for even handling like modification, deleation and creation
                           ## this is useful for APIs
## Shutil : Used to copy file from one to other.

In [3]:
# Database configuration 
db_config = {
    'user': 'root',         
    'password': 'root', 
    'host': 'localhost',               
    'database': 'file_upload_db',      
}

In [ ]:
# File Listener Handler
class FileHandler(FileSystemEventHandler):
    def __init__(self, db_connection):
        self.db_connection = db_connection
        self.fallback_directory = r"C:\Temp"  # Fallback directory for files with access issues

    def on_created(self, event):
        if not event.is_directory:
            try:
                file_path = event.src_path
                filename = os.path.basename(file_path)

                # Check if the file is accessible
                if not os.access(file_path, os.R_OK):
                    print(f"Permission error: Cannot read file '{file_path}'. Attempting to move to fallback directory.")
                    file_path = self.move_to_fallback(file_path)
                
                # Attempt to open and read the file
                with open(file_path, 'rb') as file:
                    file_content = file.read()

                # Insert file into the database
                cursor = self.db_connection.cursor()
                query = "INSERT INTO uploaded_files (filename, content) VALUES (%s, %s)"
                cursor.execute(query, (filename, file_content))
                self.db_connection.commit()

                print(f"File '{filename}' uploaded to the database.")

            except PermissionError as pe:
                print(f"Permission error: {pe}")
            except FileNotFoundError as fnfe:
                print(f"File not found: {fnfe}")
            except IOError as ioe:
                print(f"IO error occurred: {ioe}")
            except mysql.connector.Error as db_err:
                print(f"Database error: {db_err}")
            except Exception as e:
                print(f"Unexpected error occurred while uploading file '{filename}': {str(e)}")

    def move_to_fallback(self, file_path):
        """Move the file to a fallback directory if there are access issues."""
        try:
            if not os.path.exists(self.fallback_directory):
                os.makedirs(self.fallback_directory)
            new_path = os.path.join(self.fallback_directory, os.path.basename(file_path))
            shutil.move(file_path, new_path)
            print(f"File moved to fallback directory: {new_path}")
            return new_path
        except Exception as e:
            raise PermissionError(f"Failed to move file to fallback directory: {e}")

def main():
    # Establish database connection
    try:
        db_connection = mysql.connector.connect(**db_config)
        print("Connected to the database.")
    except mysql.connector.Error as err:
        print(f"Error: {err}")
        return

    # Directory to monitor
    directory_to_watch = r"C:\Users\harsh\OneDrive\Desktop\python1"

    # Setup file listener
    event_handler = FileHandler(db_connection)
    observer = Observer()
    observer.schedule(event_handler, path=directory_to_watch, recursive=False)

    # Start monitoring
    observer.start()
    print(f"Monitoring directory: {directory_to_watch}")

    try:
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        observer.stop()
        print("Stopping file listener...")

    observer.join()
    db_connection.close()

if __name__ == "__main__":
    main()

Connected to the database.
Monitoring directory: C:\Users\harsh\OneDrive\Desktop\python1
Permission error: [Errno 13] Permission denied: 'C:\\Users\\harsh\\OneDrive\\Desktop\\python1\\New Text Document.txt'
Permission error: [Errno 13] Permission denied: 'C:\\Users\\harsh\\OneDrive\\Desktop\\python1\\New Text Document.txt'
Permission error: [Errno 13] Permission denied: 'C:\\Users\\harsh\\OneDrive\\Desktop\\python1\\New Microsoft PowerPoint Presentation.pptx'
Permission error: [Errno 13] Permission denied: 'C:\\Users\\harsh\\OneDrive\\Desktop\\python1\\New Microsoft Excel Worksheet.xlsx'
Permission error: [Errno 13] Permission denied: 'C:\\Users\\harsh\\OneDrive\\Desktop\\python1\\~ew Microsoft Excel Worksheet.tmp'
File 'New Microsoft Excel Worksheet.xlsx~RF19a17278.TMP' uploaded to the database.
Permission error: [Errno 13] Permission denied: 'C:\\Users\\harsh\\OneDrive\\Desktop\\python1\\New Microsoft Excel Worksheet (2).xlsx'
Permission error: [Errno 13] Permission denied: 'C:\\User

#### Goal
###### Our script is tasked with monitoring a specific derectory for newly files.    The goal is to uploading file to the database.

###### Our scripts is like doing the work as guarding a Specific folder, like Guardian.   Whenever a new file appears , then gently picks it up and carries it to a safe place(Backup Folder). I mean in the database.  Database celebrates each new arrival , never takes his eyes off from new files.